# Base Dependencies and Libraries

In [ ]:
!pip install pymongo

In [ ]:
!pip install dnspython
!pip install geopy
!pip install folium

In [ ]:
import pandas as pd
import requests
import keys
import json

# Get JSON data format from STATCAN REST API

In [ ]:
def data_request_statcan(dimension_1, dimension_2, dimension_3, start_period, end_period):
    data_table = 'DF_17100005/'
    base_url = 'https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/data/'
    # Construct the URL with proper query parameters for period
    url = f"{base_url}{data_table}{dimension_1}.{dimension_2}.{dimension_3}?startPeriod={start_period}&endPeriod={end_period}&detail=dataonly"
    print(url)
    try:
        response = requests.get(url, headers={'Accept': 'application/json'})
        response.raise_for_status()  # Raises an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        return None
    except Exception as err:
        print(f"An error occurred: {err}")
        return None
    return response

In [ ]:
#Create dataframe with distinct provinces in Canada
province_codes = {
    'AB': 'Alberta',
    'BC': 'British Columbia',
    'MB': 'Manitoba',
    'NB': 'New Brunswick',
    'NL': 'Newfoundland and Labrador',
    'NT': 'Northwest Territories',
    'NS': 'Nova Scotia',
    'NU': 'Nunavut',
    'ON': 'Ontario',
    'PE': 'Prince Edward Island',
    'QC': 'Quebec',
    'SK': 'Saskatchewan',
    'YT': 'Yukon',

}
# Convert the dictionary into a list of tuples
province_list = list(province_codes.items())
# Create a DataFrame
df_provinces = pd.DataFrame(province_list, columns=['Code', 'Province'])

# Create Atlas Client  on MongoDB

In [ ]:
from pymongo import MongoClient

conn = MongoClient()
atlas_client = MongoClient(keys.mongo_connection_string)

### Create Database object representing the statscan database

In [ ]:
db_statscan = atlas_client.statscan

In [ ]:
#Create a collection (table) in your db_statscan object,
#Please select a meaningful name based on the data generated from create_dataframe

collection_name = 'Assignment' #choose a name
collection = db_statscan[collection_name]

# Function to request data to REST API and convert to dataframe

In [ ]:
def create_dataframe(response):
    if not response:
        return pd.DataFrame()  # Return an empty DataFrame if there was an error in the response

    data = response.json()
    data_sets = data['dataSets']
    data_structure = data['structure']['dimensions']['series']
    #print(data_structure)
    data_structure_time = data['structure']['dimensions']['observation']

    # Extract age group and time period data
    age_group_data = next(item for item in data_structure if item['name'] == 'Age group')
    age_group_values = pd.json_normalize(age_group_data, 'values')['name'].tolist()

    time_data = next(item for item in data_structure_time if item['name'] == 'Time')
    date_values = pd.json_normalize(time_data, 'values')['name'].tolist()

    # Extract the 'Geography' and 'Gender' values
    geography_value = next(item for item in data_structure if item['id'] == 'Geography')['values'][0]['name']
    sex_value = next(item for item in data_structure if item['id'] == 'Gender')['values'][0]['name']


    # Initialize the data storage with dates as the keys
    df_data = {date: [None] * len(age_group_values) for date in date_values}
    columns = ['Year'] + age_group_values

    # Map series keys to the corresponding age group index
    series_key_to_index = {series_key: index for index, series_key in enumerate(data_sets[0]['series'].keys())}

    # Process the observations to populate the data storage
    for series_key, series_data in data_sets[0]['series'].items():
        observations = series_data['observations']
        series_index = series_key_to_index[series_key]
        for key, value in observations.items():
            date = date_values[int(key)]
            df_data[date][series_index] = value[0]  # Assign the value to the correct age group

    # Convert the data storage to a DataFrame
    df_data_list = [[date] + values for date, values in df_data.items()]
    df = pd.DataFrame(df_data_list, columns=columns)

    # Add 'Geography' and 'Gender' columns with the extracted values
    df['Geography'] = geography_value
    df['Gender'] = sex_value

    # Replace missing values with NaN
    df.replace(to_replace=[None], value=pd.NA, inplace=True)

    return df

# Modify your request to STATCAN and get REST API string call

Dimension 1 = Geography
<br>
Dimension 2 = Sex (Gender)
<br>
Dimension 3 = Age Group
<br>
Start and End periods are in Years (YYYY)

In [ ]:
#Please enter the codes for each dimension in string format '' separated by ,
dimension_1_codes = ['1']
dimension_2_codes = ['1','2','3']
dimension_3_codes = ['1','25','79','80']

#Please enter the start and end periods in string format ''
start_period = '2003'
end_period = '2023'

for dimension_1 in dimension_1_codes:
    for dimension_2 in dimension_2_codes:
        for dimension_3 in dimension_3_codes:
            # Request to the STATCAN Rest API
            response = data_request_statcan(dimension_1, dimension_2, dimension_3, start_period, end_period)

            # Convert the API response into a DataFrame
            dataframe_resp = create_dataframe(response)
            dataframe_resp.head()

            # Convert the DataFrame into a list of dictionaries
            records = dataframe_resp.to_dict('records')

            # Insert the records into the MongoDB collection
            try:
                collection.insert_many(records)
                print(f"Data inserted into collection {collection_name} successfully.")
            except Exception as e:
                print(f"An error occurred: {e}")

https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/data/DF_17100005/1.1.1?startPeriod=2003&endPeriod=2023&detail=dataonly
Data inserted into collection Assignment successfully.
https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/data/DF_17100005/1.1.25?startPeriod=2003&endPeriod=2023&detail=dataonly
Data inserted into collection Assignment successfully.
https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/data/DF_17100005/1.1.79?startPeriod=2003&endPeriod=2023&detail=dataonly
Data inserted into collection Assignment successfully.
https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/data/DF_17100005/1.1.80?startPeriod=2003&endPeriod=2023&detail=dataonly
Data inserted into collection Assignment successfully.
https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/data/DF_17100005/1.2.1?startPeriod=2003&endPeriod=2023&detail=dataonly
Data inserted into collection Assignment successfully.
https://www150.statcan.gc.ca/t1/wds/sdmx/statcan/rest/data/DF_17100005/1.2.25?startPeriod=2003&en

### Search in the MongoDB collection for the max number of people per sex per province for 'All ages'

In [ ]:



def query_collection_db(sex_categories, df_provinces):
    # List to store max counts for each province, sex category, and age group
    province_sex_max_counts = []

    # Iterate over the provinces in the DataFrame
    for province in df_provinces['Province']:
        for sex in sex_categories:
            # Aggregate to find the max counts for each age group for each sex in each province
            max_count = collection.aggregate([
                {'$match': {'Geography': province, 'Gender': sex}},
                {'$project': {
                    'All ages': {'$toInt': '$All ages'},  # Convert 'All ages' to an integer if it's a string
                    '15 to 19 years': {'$toInt': '$15 to 19 years'},  # Convert '15 to 19 years' to an integer if it's a string
                    '60 to 65 years': {'$toInt': '$60 to 65 years'},  # Convert '60 to 65 years' to an integer if it's a string
                    'Geography': 1,  # Include Geography for grouping
                    'Gender': 1  # Include Sex for grouping
                }},
                {'$group': {
                    '_id': {'Gender': '$Gender'},  # Group by Sex
                    'maxAllAges': {'$max': '$All ages'},  # Get max of 'All ages'
                    'max15to19': {'$max': '$15 to 19 years'},  # Get max of '15 to 19 years'
                    'max60to65': {'$max': '$60 to 65 years'},  # Get max of '60 to 65 years'
                }}
            ])

            # Get the result
            result = next(max_count, None)
            # If there's a result, calculate population differences and append them to the list with the province name, sex, and age groups
            if result:
                maxAllAges = result['maxAllAges']
                max15to19 = result['max15to19']
                max60to65 = result['max60to65']

                # Calculate population differences
                if maxAllAges is not None and max15to19 is not None:
                    population_difference_all_ages_vs_15to19 = maxAllAges - max15to19
                else:
                    population_difference_all_ages_vs_15to19 = None

                if max15to19 is not None and max60to65 is not None:
                    population_difference_15to19_vs_60to65 = max15to19 - max60to65
                else:
                    population_difference_15to19_vs_60to65 = None

                province_sex_max_counts.append({
                    'Province': province,
                    'Gender': result['_id']['Gender'],
                    'maxAllAges': maxAllAges,
                    'max15to19': max15to19,
                    'max60to65': max60to65,
                    'population_difference_all_ages_vs_15to19': population_difference_all_ages_vs_15to19,
                    'population_difference_15to19_vs_60to65': population_difference_15to19_vs_60to65
                })
            else:
                # If no result, append a dict with 'None' for all values
                province_sex_max_counts.append({
                    'Province': province,
                    'Gender': sex,
                    'maxAllAges': None,
                    'max15to19': None,
                    'max60to65': None,
                    'population_difference_all_ages_vs_15to19': None,
                    'population_difference_15to19_vs_60to65': None
                })

    # Convert the list of max counts to a DataFrame
    df_max_counts = pd.DataFrame(province_sex_max_counts)
    df_max_counts.fillna(0, inplace=True)  # Fill NaN values with 0 for better presentation

    return df_max_counts


In [ ]:
# Define the sex categories to iterate over
sex_categories = ['Men+', 'Women+', 'Total - gender']
df_province_maxcount = query_collection_db(sex_categories, df_provinces)

In [ ]:
df_province_maxcount.head()

,Province,Gender,maxAllAges,max15to19,max60to65,population_difference_all_ages_vs_15to19,population_difference_15to19_vs_60to65
0,Alberta,Men+,2363694.0,145710.0,0,2217984.0,0
1,Alberta,Women+,2331596.0,136571.0,0,2195025.0,0
2,Alberta,Total - gender,4695290.0,282281.0,0,4413009.0,0
3,British Columbia,Men+,2736718.0,150078.0,0,2586640.0,0
4,British Columbia,Women+,2782295.0,142130.0,0,2640165.0,0


# Import libraries and province_codes dictionary to use MapQuest geocode

In [ ]:
import time
from geopy.geocoders import MapQuest
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
from requests.exceptions import ConnectionError  # Import the ConnectionError
from province_codes import province_codes
from geopy.location import Location
import folium

### Create map by province

In [ ]:
# Initialize the geocoder with your API key.
geo = MapQuest(api_key=keys.mapquest_key)
province_list = df_province_maxcount.Province.unique()
province_list.sort()
province_list = province_list.tolist()

# Reverse the province_codes dictionary
province_codes_reversed = {v: k for k, v in province_codes.items()}

locations = []
for province_name in province_list:
    processed = False
    delay = .1  # Initial delay before retrying in seconds.
    while not processed:
        try:
            province_code = province_codes_reversed[province_name]
            location = geo.geocode(f'{province_code}, Canada')
            if location:
                location = Location(address=f'{province_name},{location.address}',
                                    point=location.point,
                                    raw=location.raw)
            locations.append(location)
            print(location)
            processed = True
        except GeocoderTimedOut:
            print('Geocoder service timed out. Waiting before retrying...')
            time.sleep(delay)
            delay += .1  # Increment the delay
        except GeocoderServiceError as e:
            print(f'Geocoder service error: {e}')
            break  # Exit the loop or handle it in some other way.
        except ConnectionError as e:
            print(f'Connection error occurred: {e}')
            print('Will retry after delay.')
            time.sleep(delay)
            delay *= 2  # Use exponential backoff for the retry delay
        except Exception as e:
            print(f'An unexpected error occurred: {e}')
            break  # Exit the loop or handle it in some other way.

Alberta,AB, CA
British Columbia,BC, CA
Manitoba,MB, CA
New Brunswick,NB, CA
Newfoundland and Labrador,NL, CA
Northwest Territories,NT, CA
Nova Scotia,NS, CA
Nunavut,NU, CA
Ontario,ON, CA
Prince Edward Island,PE, CA
Quebec,QC, CA
Saskatchewan,SK, CA
Yukon,YT, CA


In [ ]:
# Find the index of the maximum value in the 'max15to19' column for each province
idx = df_province_maxcount.groupby('Province')['max15to19'].idxmax()

# Select the rows corresponding to the indices
max_by_province = df_province_maxcount.loc[idx]




In [ ]:
# Creating a base map
canadamap = folium.Map(location=[43.653963, -79.387207], zoom_start=4, detect_retina=True)

#Create markers
sorted_df = df_province_maxcount.sort_values(by='max15to19', ascending=False)

# Create a dictionary to easily map provinces to their locations
location_dict = {loc.address.split(",")[0]: loc for loc in locations}

# Iterate over the groups in sorted dataframe
for name, group in sorted_df.groupby('Province'):
    # Check if the province name is in the location dictionary
    if name in location_dict:
        # Extract the location object
        loc = location_dict[name]
        # Start with the province code in the popup text
        popup_text = [f"{province_codes_reversed[name]}"]
        # Append maxAllAges for each sex in this province
        for row in group.itertuples():
            popup_text.append(f"{row.Gender}: {row.max15to19}")
        # Join the popup lines with HTML break line
        popup_html = '<br>'.join(popup_text)
        # Create a marker with the popup HTML
        marker = folium.Marker(
            tooltip='Max Total People by Province',
            location=[loc.latitude, loc.longitude],
            popup=folium.Popup(popup_html,parse_html=False)
        )
        # Add the marker to the map
        marker.add_to(canadamap)
    else:
        print(f"Location for {name} not found in locations list.")

# Display the map
canadamap.save('Population_maxALLages.html')  # Saves the map to an HTML file which you can open in a browser. Replace the #’s with the name of your map.
